In [2]:
from langchain_huggingface import (
    HuggingFaceEndpoint, ChatHuggingFace, HuggingFaceEmbeddings
)
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema


In [37]:
rewrite_3_prompt = '''
<s><INST>You are an expert in rephrasing complex employee-related questions into short, compressed, and easy-to-understand alternatives.
Given a user’s original question, rewrite it into 3 distinct simplified queries that retain the core meaning but are easier to process and interpret.

Each rewritten query should:
- Be short and direct (1–2 lines)
- Avoid unnecessary words
- Use simpler language
- Capture the original intent accurately

{format_instructions}

Question: {input_question}
Answer:</INST>
'''.strip()
schema = [
    ResponseSchema(
        name='answer',
        description='List of 3 re-written queries that are short and easy to understand.',
        type='list'
    )
]
parser = StructuredOutputParser.from_response_schemas(schema)
format_instructions = parser.get_format_instructions()

human_in_loop_prompt=PromptTemplate(
    template=rewrite_3_prompt,
     input_variables=['input_question'],
    partial_variables={"format_instructions": format_instructions}
)

parser=StrOutputParser()
llm = HuggingFaceEndpoint(
repo_id="meta-llama/Llama-3.2-3B-Instruct",
task="text-generation"
)
model = ChatHuggingFace(llm=llm)

chain=(        
        human_in_loop_prompt
        | model
        | parser
    )
response=chain.invoke({
    'input_question':'get me the details of unis'
})

In [31]:
import json
from ast import literal_eval
import re
# result=literal_eval(response)

fixed = re.sub(r'("\s*[^"]+?\?)(?="\s*")', r'\1,",', response)

In [41]:
print(response)

```json
{
  "answer": [
    "What type of university information are you looking for?",
    "Can you please specify what details of universities you need?",
    "What specific information about universities do you require?"
  ]
}
```

These rewritten queries aim to capture the essence of the original question, 'get me the details of unis', while making it concise and easy to understand.


In [34]:
json.loads(fixed)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# experiment 1

In [56]:
from pydantic import BaseModel,Field

class RewriteQueryOutputFormat(BaseModel):
  query:list=Field(...,description='The re-written user query')


rewrite_3_prompt = '''
You are an expert in rephrasing complex employee-related questions into short, compressed, and easy-to-understand alternatives.
Given a user’s original question, rewrite it into 3 distinct simplified queries that retain the core meaning but are easier to process and interpret.

Each rewritten query should:
- Be short and direct (1–2 lines)
- Avoid unnecessary words
- Use simpler language
- Capture the original intent accurately

{format_instructions}

Question: {input_question}
Answer:
'''.strip()


schema = [
    ResponseSchema(
        name='answer',
        description='List of 3 re-written queries that are short and easy to understand.'
    )
]

parser = StructuredOutputParser.from_response_schemas(schema)
format_instructions = parser.get_format_instructions()


prompt=PromptTemplate(
    template=rewrite_3_prompt,
     input_variables=['input_question'],
    partial_variables={"format_instructions": format_instructions}
)

In [57]:
llm = HuggingFaceEndpoint(
repo_id="meta-llama/Llama-3.2-3B-Instruct",
task="text-generation"
)
model = ChatHuggingFace(llm=llm)



In [58]:
chain=(
    prompt
    | model
    | parser
)


In [59]:
response=chain.invoke({
    "input_question":"pull me all the details of the person by the name unis and also the persons joining date"
})

In [60]:
print(response)

{'answer': ["What is the person's details by name Unis?", 'Who is Unis in our database?', "Can you show Unis's profile?"]}


# human in loop

In [8]:
llm = HuggingFaceEndpoint(
repo_id="meta-llama/Llama-3.2-3B-Instruct",
task="text-generation"
)
# parser=StrOutputParser()
model = ChatHuggingFace(llm=llm)

chain=(        
        rewrite_3_prompt
        | model
    )
chain.invoke('get me the details of unis')

c:\Users\Lijin Durairaj\AppData\Local\conda\conda\envs\modelenvnew\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>

# langraph

In [1]:
from langchain_core.runnables.graph import StateGraph, START, END
from langchain_core.runnables import RunnableLambda
from typing import TypedDict

# Input schema
class Person(TypedDict):
    name: str
    age: int

# Nodes
def greet(state: Person):
    print(f"Hello, {state['name']}!")
    return state

def check_age(state: Person):
    if state["age"] >= 21:
        return "can_drink"
    elif state["age"] >= 16:
        return "can_drive"
    else:
        return "minor"

def can_drink(state: Person):
    print("You can legally drink 🍺")
    return state

def can_drive(state: Person):
    print("You can drive 🚗")
    return state

def minor(state: Person):
    print("You're a minor 🚫")
    return state

# Build graph
graph = StateGraph(Person)

graph.add_node("greet", greet)
graph.add_node("check_age", RunnableLambda(check_age))  # ✅ wrapped as RunnableLambda
graph.add_node("can_drink", can_drink)
graph.add_node("can_drive", can_drive)
graph.add_node("minor", minor)

graph.add_edge(START, "greet")
graph.add_edge("greet", "check_age")

# ✅ Correct conditional edge mapping
graph.add_conditional_edges(
    "check_age",
    {
        "can_drink": "can_drink",
        "can_drive": "can_drive",
        "minor": "minor"
    }
)

graph.add_edge("can_drink", END)
graph.add_edge("can_drive", END)
graph.add_edge("minor", END)

# Compile
app = graph.compile()

ImportError: cannot import name 'StateGraph' from 'langchain_core.runnables.graph' (c:\Users\Lijin Durairaj\AppData\Local\conda\conda\envs\modelenvnew\Lib\site-packages\langchain_core\runnables\graph.py)